# Shopping During Covid-19 LockDown in South Africa

As South Africa is on Day 2 of a 21-Day National Lock-down to stop the spread of the novel Corona Virus, I have thought it appropriate to include a Lock-down theme in my project.

In South Africa (and a few other places around the world, we are only allowed to buy groceries and get health care.  One of my (and many others') concerns are that many people probably already have the virus and not even know it, and are therefore spreading it around when they do shopping.  
If I could choose, I would rather go to a grocery store which is not near a pharmacy, as a lot of people might want to go to both on their limited outings.  
If I could figure out which grocery stores are further away from pharmacies and vise versa, I could assume that those stores might have less foot traffic and less chances of getting infected.  
I can also add health care facilities where one would assume people go when they suspect that they are ill.  If they are ill, but their symptoms are not severe enough to be admitted to hospital, they might stop by a nearby store or pharmacy on their way home to stock up on food and medicine for their quarantine period.  
For people who want to stay away from other people while shopping, they might also avoid stores and pharmacies near hospitals.

The target audience for this project will be anyone who is looking for the best places to buy their essentials during this lock-down period. 
Some, like me, might want to go to isolated stores where there is a chance of less people.  Yet others might want to go to the closest stores they can get, and maybe where grocery stores and pharmacies are close together.  
Either way, this projects will help with that.

In [76]:
import pandas as pd
import numpy as np

In [77]:
conda update -n base -c defaults conda

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - conda


The following packages will be UPDATED:

  conda                                        4.8.2-py37_0 --> 4.8.3-py37_0


Preparing transaction: ...working... done
Verifying transaction: ...working... failed

Note: you may need to restart the kernel to use updated packages.



EnvironmentNotWritableError: The current user does not have write permissions to the target environment.
  environment location: C:\ProgramData\Anaconda3




I Install folium and geopy so that I will be able to draw a map, as well as plot different coordinates on it.

In [78]:
!pip install geopy

In [79]:
!pip install folium

In [80]:
import requests
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

## Now I define my Foursquare Credentials and Version

In [81]:
#Establishing FourSquare Credentials
CLIENT_ID = 'AKBWLJHMGEQFFNUQ0AZYFLMKNQI0LUL4BWVVX43TMR0MCO5N' # your Foursquare ID
CLIENT_SECRET = 'KZEMDT2LQ3SPMR0H13IUQUDPMMMN24KEVEJMUXT2FM2DKNEB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AKBWLJHMGEQFFNUQ0AZYFLMKNQI0LUL4BWVVX43TMR0MCO5N
CLIENT_SECRET:KZEMDT2LQ3SPMR0H13IUQUDPMMMN24KEVEJMUXT2FM2DKNEB


In [89]:
address = '6 Olympus Street, Florida Hills, Gauteng, South Africa'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-26.1670617 27.925278


# Grocery Stores

During the LockDown period, we are only allowed to buy essential items.  These items can be bought at grocery stores.  To cover more venues, I used the Category ID's for Grocery Stores, Convenience Stores, Super Markets, Fruit and Vegetable stores and Butchers

Put your Coordinates in at the en of the URL

In [92]:
urlFood = "https://api.foursquare.com/v2/venues/explore?categoryId=4bf58dd8d48988d118951735,4d954b0ea243a5684a65b473,52f2ab2ebcbc57f1066b8b46,52f2ab2ebcbc57f1066b8b1c,4bf58dd8d48988d11d951735&client_id=AKBWLJHMGEQFFNUQ0AZYFLMKNQI0LUL4BWVVX43TMR0MCO5N&client_secret=KZEMDT2LQ3SPMR0H13IUQUDPMMMN24KEVEJMUXT2FM2DKNEB&v=20180605&radius=10000&limit=100&ll=-26.1618,%2027.9255"
Groceries_json = requests.get(urlFood).json()
Groceries_json

{'meta': {'code': 200, 'requestId': '5e7f63300de0d9001ba801e8'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'query': 'grocery store',
  'totalResults': 98,
  'suggestedBounds': {'ne': {'lat': -26.07179990999991,
    'lng': 28.025585562255472},
   'sw': {'lat': -26.25180009000009, 'lng': 27.825414437744527}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e197c6914957dc705e17cac',
       'name': 'Meat World',
       'location': {'address': '14th Ave',
        'crossStreet': 'Bagley Terrace',
        'lat': -26.14445916759537,
        'lng': 27.954662118046485,
        'l

The next step is to clean the Json file and create a dataframe with columns and rows.

In [83]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = Groceries_json['response']['groups'][0]['items']
    
Grocery_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Grocery_venues =Grocery_venues.loc[:, filtered_columns]

# filter the category for each row
Grocery_venues['venue.categories'] = Grocery_venues.apply(get_category_type, axis=1)

# clean columns
Grocery_venues.columns = [col.split(".")[-1] for col in Grocery_venues.columns]

Grocery_venues

,name,categories,lat,lng
0,Meat World,Butcher,-26.144459,27.954662
1,Florida Junction SUPERSPAR,Professional & Other Places,-26.148534,27.901698
2,Impala Fruit And Veg,Fruit & Vegetable Store,-26.144368,27.981086
3,Steve's Spar,Grocery Store,-26.136481,27.976233
4,Checkers Hyper - Constantia Kloof,Grocery Store,-26.147755,27.921493
...,...,...,...,...
93,Engen Mintys Tyres & Mags Amalgam,Gas Station,-26.212846,27.999331
94,Engen Tahero Convenience Centre,Gas Station,-26.156567,27.832421
95,SPAR Witpoortjie,Deli / Bodega,-26.133421,27.834431
96,SPAR Blairgowrie,Deli / Bodega,-26.115769,28.010941


## Map of all the Grocery Stores in the area (Blue Dots)

I use Geocode and Folium to draw up a map and pinpoint locations. All the locations for Grocery stores are pinned in Blue.

In [93]:
GroceryShop = folium.Map(location=[-26.1618,27.9255],zoom_start=13)
for lat, lng, name, categories in zip(Grocery_venues['lat'], Grocery_venues['lng'], Grocery_venues['name'], Grocery_venues['categories']):
    label = '{}, {}'.format(categories, name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(GroceryShop) 
    
GroceryShop

# Pharmacies

For this Foursquare search, I used the Category ID's for Pharmacies, Drug Stores and medical Supply Stores

In [84]:
urlMeds = "https://api.foursquare.com/v2/venues/explore?categoryId=5745c2e4498e11e7bccabdbd,4bf58dd8d48988d10f951735,58daa1558bbb0b01f18ec206&client_id=AKBWLJHMGEQFFNUQ0AZYFLMKNQI0LUL4BWVVX43TMR0MCO5N&client_secret=KZEMDT2LQ3SPMR0H13IUQUDPMMMN24KEVEJMUXT2FM2DKNEB&v=20180605&radius=10000&limit=100&ll=-26.1618,%2027.9255"
BuyMedicine_json = requests.get(urlMeds).json()
BuyMedicine_json

{'meta': {'code': 200, 'requestId': '5e7f47daad1ab4001bac91c2'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'query': 'drugstore',
  'totalResults': 45,
  'suggestedBounds': {'ne': {'lat': -26.07179990999991,
    'lng': 28.025585562255472},
   'sw': {'lat': -26.25180009000009, 'lng': 27.825414437744527}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d1067689302a35dc9e27a6e',
       'name': 'Dis-Chem',
       'location': {'address': '1724 Christiaan de Wet Road',
        'crossStreet': 'Christiaan De Wet Rd. & Hendrik Potgieter Rd.',
        'lat': -26.126857967380193,


The next step is to clean the Json file and create a dataframe with columns and rows.

In [85]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
Medvenues = BuyMedicine_json['response']['groups'][0]['items']
    
Pharmacy_venues = pd.json_normalize(Medvenues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Pharmacy_venues =Pharmacy_venues.loc[:, filtered_columns]

# filter the category for each row
Pharmacy_venues['venue.categories'] = Pharmacy_venues.apply(get_category_type, axis=1)

# clean columns
Pharmacy_venues.columns = [col.split(".")[-1] for col in Pharmacy_venues.columns]

Pharmacy_venues

,name,categories,lat,lng
0,Dis-Chem,Pharmacy,-26.126858,27.904855
1,Dis-Chem,Pharmacy,-26.115425,27.896948
2,The Local Choice Pharmacy Park,Pharmacy,-26.159440,27.904598
3,Medicare Pharmacy,Pharmacy,-26.150692,27.943865
4,The Local Choice Pharmacy El Corro,Pharmacy,-26.144580,27.954464
5,Clicks,Cosmetics Shop,-26.136924,27.918601
6,Medicare Pharmacy Palm Court,Pharmacy,-26.125783,27.932122
7,Dis-Chem,Pharmacy,-26.137964,28.008667
8,Clicks,Cosmetics Shop,-26.127674,27.903798
9,Dis-Chem,Pharmacy,-26.139734,27.978989


## Map of all the Pharmacies in the area (Red dots)

I use Geocode and Folium to draw up a map and pinpoint locations. All the locations for Pharmacies are pinned in Red

In [94]:
Pharmacies = folium.Map(location=[-26.1618,27.9255],zoom_start=13)
for lat, lng, name, categories in zip(Pharmacy_venues['lat'], Pharmacy_venues['lng'], Pharmacy_venues['name'], Pharmacy_venues['categories']):
    label = '{}, {}'.format(categories, name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#f58e8e',
        fill_opacity=0.7,
        parse_html=False).add_to(Pharmacies) 

Pharmacies

# Hospitals

For this Foursquare search, I used the Category ID's for Hospitals, Emergency Rooms, Doctors offices and Urgent Care Centers

In [86]:
urlHOSP = "https://api.foursquare.com/v2/venues/explore?categoryId=4bf58dd8d48988d194941735,4bf58dd8d48988d196941735,4bf58dd8d48988d177941735,56aa371be4b08b9a8d573526&client_id=AKBWLJHMGEQFFNUQ0AZYFLMKNQI0LUL4BWVVX43TMR0MCO5N&client_secret=KZEMDT2LQ3SPMR0H13IUQUDPMMMN24KEVEJMUXT2FM2DKNEB&v=20180605&radius=10000&limit=100&ll=-26.1618,%2027.9255"
Hospitals_json = requests.get(urlHOSP).json()
Hospitals_json

{'meta': {'code': 200, 'requestId': '5e7f48b1df2774001bb8024e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'query': 'emergency room',
  'totalResults': 23,
  'suggestedBounds': {'ne': {'lat': -26.07179990999991,
    'lng': 28.025585562255472},
   'sw': {'lat': -26.25180009000009, 'lng': 27.825414437744527}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ba209c7f964a520bad737e3',
       'name': 'Life Flora Clinic',
       'location': {'address': 'Constantia Blvd',
        'lat': -26.151518403688453,
        'lng': 27.92002558708191,
        'labeledLatLngs': [{'label':

The next step is to clean the Json file and create a dataframe with columns and rows.

In [87]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
Hospvenues = Hospitals_json['response']['groups'][0]['items']
    
Hospital_venues = pd.json_normalize(Hospvenues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Hospital_venues =Hospital_venues.loc[:, filtered_columns]

# filter the category for each row
Hospital_venues['venue.categories'] = Hospital_venues.apply(get_category_type, axis=1)

# clean columns
Hospital_venues.columns = [col.split(".")[-1] for col in Hospital_venues.columns]

Hospital_venues

,name,categories,lat,lng
0,Life Flora Clinic,Hospital,-26.151518,27.920026
1,Mayo Clinic,Doctor's Office,-26.150655,27.920572
2,Medicross Randburg,Medical Center,-26.110720,27.989679
3,Strubensvalley Medical Centre,Doctor's Office,-26.120884,27.901995
4,Dr Chane Magnus,Doctor's Office,-26.135487,27.969423
5,Rahima Moosa Mother and Child Hospital,Hospital,-26.189145,27.971888
6,Triomf Medicross,Doctor's Office,-26.174927,27.979233
7,Northcliff Health Centre,Doctor's Office,-26.137718,27.977248
8,Dr. Dorianne Green,Doctor's Office,-26.146000,27.984500
9,Intercare Wilgeheuwel,Doctor's Office,-26.113402,27.893615


## Map of all the Medical centers in the area (Yellow dots)

I use Geocode and Folium to draw up a map and pinpoint locations. All the locations for Hospitals are pinned in Yellow

In [95]:
Medical = folium.Map(location=[-26.1618,27.9255],zoom_start=13)

for lat, lng, name, categories in zip(Hospital_venues['lat'], Hospital_venues['lng'], Hospital_venues['name'], Hospital_venues['categories']):
    label = '{}, {}'.format(categories, name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#fed37e',
        fill_opacity=0.7,
        parse_html=False).add_to(Medical) 
    
Medical

# Location Map

I use Geocode and Folium to draw up a map and pinpoint locations.  All the locations for Grocery stores are pinned in Blue.  Locations for Pharmacies are pinned in Red. And Locations for Hospitals are pinned in Yellow

In [88]:
EssentialShop = folium.Map(location=[-26.1618,27.9255],zoom_start=13)
for lat, lng, name, categories in zip(Grocery_venues['lat'], Grocery_venues['lng'], Grocery_venues['name'], Grocery_venues['categories']):
    label = '{}, {}'.format(categories, name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(EssentialShop) 
for lat, lng, name, categories in zip(Pharmacy_venues['lat'], Pharmacy_venues['lng'], Pharmacy_venues['name'], Pharmacy_venues['categories']):
    label = '{}, {}'.format(categories, name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#f58e8e',
        fill_opacity=0.7,
        parse_html=False).add_to(EssentialShop) 
for lat, lng, name, categories in zip(Hospital_venues['lat'], Hospital_venues['lng'], Hospital_venues['name'], Hospital_venues['categories']):
    label = '{}, {}'.format(categories, name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#fed37e',
        fill_opacity=0.7,
        parse_html=False).add_to(EssentialShop) 
    
EssentialShop